In [1]:
! pip install transformers

     |████████████████████████████████| 2.0MB 19.3MB/s 
     |████████████████████████████████| 870kB 55.0MB/s 
     |████████████████████████████████| 3.3MB 52.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=6dc3699fc86ee63d81bb8ff8a45f87f4bad1e66af3008b665ce0ab3fcaf6203b
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
cd drive/MyDrive/LXPER

/content/drive/MyDrive/LXPER


In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter="\t", header=None)[:2000]
test_data = pd.read_csv("./cola_public/raw/in_domain_dev.tsv", delimiter="\t", header=None)[:2000]

In [4]:
from transformers import OpenAIGPTForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification
import torch
use_cuda=True

num_labels=2
model = OpenAIGPTForSequenceClassification.from_pretrained('openai-gpt', num_labels=num_labels)
if use_cuda and torch.cuda.is_available():
    model.cuda()
model.train()

Some weights of OpenAIGPTForSequenceClassification were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OpenAIGPTForSequenceClassification(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(40478, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, e

In [5]:
model.transformer.tokens_embed.weight[0][:20]

tensor([ 0.0464, -0.1280, -0.0153,  0.0641, -0.0497, -0.0210,  0.1288,  0.0593,
        -0.0790, -0.0448,  0.0073, -0.0828, -0.0677, -0.1903, -0.0698,  0.0617,
        -0.0233, -0.0672, -0.0407,  0.0749], device='cuda:0',
       grad_fn=<SliceBackward>)

In [6]:
from transformers import AdamW
import torch.nn as nn
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

In [7]:
text_batch = list(data.iloc[:,3])
labels = list(data.iloc[:,1])

from transformers import OpenAIGPTTokenizer

tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt') 
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True, add_special_tokens=True, max_length=512)

input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

print(input_ids.shape)
print(attention_mask.shape)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.



torch.Size([2000, 44])
torch.Size([2000, 44])


In [8]:
encoding['labels'] = torch.tensor(labels)

In [9]:
model.resize_token_embeddings(len(tokenizer))
model.transformer.tokens_embed.weight[0][:20]

tensor([ 0.0464, -0.1280, -0.0153,  0.0641, -0.0497, -0.0210,  0.1288,  0.0593,
        -0.0790, -0.0448,  0.0073, -0.0828, -0.0677, -0.1903, -0.0698,  0.0617,
        -0.0233, -0.0672, -0.0407,  0.0749], device='cuda:0',
       grad_fn=<SliceBackward>)

In [10]:
device = 'cuda:0'
model = model.to(device)
encoding = encoding.to(device)

In [11]:
for epoch in range(3):
    for i in range(data.shape[0]):
        input = encoding['input_ids'][i].view(1, -1)
        mask = encoding['attention_mask'][i].view(1, -1)
        label = encoding['labels'][i].view(-1)

        optimizer.zero_grad()

        output = model(input, mask, labels=label)
        loss = criterion(output.logits, label)
        print(loss)
        print('EPOCH:',epoch, round(i/data.shape[0], 4), '%')
        loss.backward()
        optimizer.step()

Streaming output truncated to the last 5000 lines.
tensor(0.3439, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.75 %
tensor(0.2238, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.7505 %
tensor(0.1017, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.751 %
tensor(0.2547, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.7515 %
tensor(0.2883, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.752 %
tensor(0.1761, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.7525 %
tensor(0.3241, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.753 %
tensor(0.1843, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.7535 %
tensor(0.2964, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.754 %
tensor(0.1393, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.7545 %
tensor(2.0974, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.755 %
tensor(0.2970, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 0.7555 %
tensor(0.2669, device='cuda:0', 

In [12]:
model.eval()

OpenAIGPTForSequenceClassification(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(40479, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, e

In [13]:
test_text = list(test_data.iloc[:,3])
test_label = list(test_data.iloc[:,1])

In [14]:
test_encoding = tokenizer(test_text, return_tensors='pt', padding=True, truncation=True)

test_encoding = test_encoding.to(device)
test_input_ids = test_encoding['input_ids']
test_attention_mask = test_encoding['attention_mask']

In [15]:
pred = []
for i in range(test_data.shape[0]):
        input = test_encoding['input_ids'][i].view(1, -1)
        mask = test_encoding['attention_mask'][i].view(1, -1)
        
        output = model(input, mask)
        output = int(output.logits.argmax())
        pred.append(output)

In [16]:
from sklearn.metrics import accuracy_score

accuracy_score(test_label, pred)

0.7533206831119544

In [17]:
torch.save(model.state_dict(), './models/gpt1_statedict')

In [ ]:
test_encoding = tokenizer(['I are a boy'], return_tensors='pt', padding=True, truncation=True).to(device)
test_input_ids = test_encoding['input_ids']
test_attention_mask = test_encoding['attention_mask']

input = test_encoding['input_ids'].view(1, -1)
mask = test_encoding['attention_mask'].view(1, -1)        
output = model(input, mask)